In [1]:
from dolfinx import fem, mesh, plot, io, geometry
from dolfinx.fem.petsc import LinearProblem
from petsc4py.PETSc import ScalarType
from mpi4py import MPI
import numpy as np
from ufl import ds, dx, grad, inner
import ufl
import matplotlib as mpl
import pyvista
from typing import List, Tuple
import sys
import os

# Get the absolute path to the src directory
src_path = os.path.abspath('..')
if src_path not in sys.path:
    sys.path.append(src_path)
from src.visualization import timeDependentVariableToGif, plot_array, plot_function
from src.solveStateEquation import solveStateEquation, getSourceTerm, buildControlFunction
from src.solveAdjointEquation import solveAdjointEquation
from src.solveFinDimObjective import solveFinDimProblem
from src.tools import getValueOfFunction, buildIterationFunction
from src.ExtremalPoints import ExtremalPoint
from src.HesseMatrix import calculateL2InnerProduct, HesseMatrix
from src.semiSmoothNewtonSolver import computeSemiNewtonStep
from dataclasses import dataclass

@dataclass
class Parameters:
    T = 1
    dt = 0.01
    x1 = (0.5, 0.5)
    x2 = (-0.5, -0.5)
    area = 4
    d = 2
    mollify_const = 0.01
    newton_c = 1
    alpha = 2.2
    beta = 2.4
    waveSpeed = 1
    randomFactor = 0.05
    solverIteration = 0
    yd = []
    msh = mesh.create_rectangle(
        comm=MPI.COMM_WORLD,
        points=((-1., -1.), (1.0, 1.0)),
        n=(32, 32),
        cell_type=mesh.CellType.triangle,
    )
    V = fem.functionspace(msh, ("Lagrange", 1))
    
params = Parameters()

Test: Evaluation of `fem.Function` with `src.tools.getValueOfFunction()`

In [2]:
test_function = lambda x: np.sin(x[0] + x[1])
obj = fem.Function(params.V)
obj.interpolate(test_function)
testPoint = (1, 1)
pointList = [testPoint]
print(testPoint, ' Real value: ', test_function(testPoint), ' Value from discrete grid: ', 
      getValueOfFunction(params.V, obj, pointList))

(1, 1)  Real value:  0.9092974268256817  Value from discrete grid:  [0.90929743]


Hesse matrix

In [3]:
point1 = ExtremalPoint(np.array([1, 0]), 0.2, 0, params)
active_set = [point1]
hesse = HesseMatrix([], params)
print('Only standard states: ', hesse.matrix)
hesse.update(active_set)
print('Appending one state: ', hesse.matrix)
point2 = ExtremalPoint(np.array([1, 0]), 0.5, 1, params)
active_set.append(point2)
hesse.update(active_set)
print('Appending two states: ', hesse.matrix)
active_set.pop(0)
point3 = ExtremalPoint(np.array([1, 0]), 0.6, 1, params)
active_set.append(point2)
hesse.update(active_set)
print('Removing one state: ', hesse.matrix)
print('Hesse is symmetric: ', np.allclose(hesse.matrix, hesse.matrix.T, rtol=1e-5, atol=1e-8))

Only standard states:  [[0.0178966  0.0178966  0.06189623 0.06189623]
 [0.0178966  0.0178966  0.06189623 0.06189623]
 [0.06189623 0.06189623 0.22036867 0.22036867]
 [0.06189623 0.06189623 0.22036867 0.22036867]]
Appending one state:  [[0.01527211 0.0165137  0.0165137  0.05744897 0.05744897]
 [0.0165137  0.0178966  0.0178966  0.06189623 0.06189623]
 [0.0165137  0.0178966  0.0178966  0.06189623 0.06189623]
 [0.05744897 0.06189623 0.06189623 0.22036867 0.22036867]
 [0.05744897 0.06189623 0.06189623 0.22036867 0.22036867]]
Appending two states:  [[1.52721104e-02 6.01011402e-04 1.65136996e-02 1.65136996e-02
  5.74489702e-02 5.74489702e-02]
 [6.01011402e-04 2.84724270e-05 6.63194422e-04 6.63194422e-04
  2.15416688e-03 2.15416688e-03]
 [1.65136996e-02 6.63194422e-04 1.78966022e-02 1.78966022e-02
  6.18962340e-02 6.18962340e-02]
 [1.65136996e-02 6.63194422e-04 1.78966022e-02 1.78966022e-02
  6.18962340e-02 6.18962340e-02]
 [5.74489702e-02 2.15416688e-03 6.18962340e-02 6.18962340e-02
  2.203686

Adjoint equation: The vectors $P_k$ and $p_k$ should have zero boundary values

In [4]:

s1 = lambda t : 0.05 if t <= 0.25 else (0.65 if t <= 0.5 else (0.15 if t <= 0.75 else 0.35))
s2 = lambda t : 0.775 if t <= 0.25 else (-0.025 if t <= 0.5 else (0.975 if t <= 0.75 else 0.275))

g1 = getSourceTerm(params.x1, params)
g2 = getSourceTerm(params.x2, params)

control = buildControlFunction([g1, g2], [s1, s2], params)
yd = solveStateEquation(control, params)

u0 = ExtremalPoint(np.array([1., 0]), 0.5, type=0, params=params, idx=2)
active_set = [u0]
hesse = HesseMatrix(active_set, params)
weights = np.ones(len(active_set))
slope = np.zeros((params.d,))
y_shift = np.zeros((params.d,))

weights, slope, y_shift = computeSemiNewtonStep(weights, slope, y_shift, active_set, hesse, params)
print(weights, slope, y_shift)